# One Shot Learning

Here we are checking the performance of the model trained on the English Dataset on other Datasets and their translated versions without any finetuning

## Imports

In [1]:
from bert import BERT
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

from data_cleaning import Data_Preprocessing
from arabert.preprocess import ArabertPreprocessor

## Utility Functions

In [2]:
def preprocess(df,isArabic):
    
    X = df['Text']
    X_new=[]
    if(isArabic):
        prep = ArabertPreprocessor('bert-base-arabertv02')
        for text in tqdm(X):
            text = prep.preprocess(text)
            X_new.append(text)
    else:
        processer = Data_Preprocessing()
        for text in tqdm(X):
            text= processer.removeEmojis(text)
            text = processer.removeUrls(text)
            text=processer.removeSpecialChar(text)
            X_new.append(text)

    df['Text']=X_new
    return df 

In [3]:
def load_dataset(args,data_path,index):
    # read dataframes
    df_test = pd.read_csv(data_path+'test_'+str(index)+'.csv')

    # clean data
    df_test=preprocess(df_test,args['isArabic'])

    return df_test

In [4]:
def one_shot_output(model_path,data_path,obj,args):
    saved_model=obj.load_model(model_path,args)
    device = torch.device(args['device'])
    saved_model=saved_model.to(device)
    
    all_metrics={}
    
    # preprocessing
    for fold in [1,2,3,4,5]:
        df = load_dataset(args,data_path,fold)

        metrics = obj.run_test(saved_model,df,args)
        
        for key,value in metrics.items():
            if(key not in all_metrics):
                all_metrics[key]=value
            else:
                all_metrics[key]+=value
    
    for key,value in all_metrics.items():
        all_metrics[key]/=5
    
    return all_metrics

### Arabic

In [5]:
DATA_PATH = "Data_Processed/Let-Mi/"
MODEL_PATH = "Saved_Models/Let-Mi/all_but_one/best_bert_bert_3_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': True,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a Be

In [6]:
metrics

{'Test_accuracy': 0.582421875,
 'Test_mF1Score': 0.5419605835011622,
 'Test_f1Score': 0.40600068873181633,
 'Test_auc': 0.5786451279075037,
 'Test_precision': 0.6827326020102771,
 'Test_recall': 0.2892143656246628,
 'Test_non_hatef1Score': 0.6779204782705082,
 'Test_non_recallScore': 0.8680758901903445,
 'Test_non_precisionScore': 0.5561452688242857,
 'Test_avg_loss': 1.535088037326932}

### Italian

In [8]:
DATA_PATH = "Data_Processed/AMI-2020/"
MODEL_PATH = "Saved_Models/AMI-2020/all_but_one/best_bert_bert_2_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [9]:
metrics

{'Test_accuracy': 0.48456504494976194,
 'Test_mF1Score': 0.4615747304513461,
 'Test_f1Score': 0.35039373161261916,
 'Test_auc': 0.4782228968700304,
 'Test_precision': 0.44805594360321443,
 'Test_recall': 0.2877535590445205,
 'Test_non_hatef1Score': 0.5727557292900729,
 'Test_non_recallScore': 0.6686922346955404,
 'Test_non_precisionScore': 0.5009207409952607,
 'Test_avg_loss': 1.2697309851142102}

### Hindi 

In [10]:
DATA_PATH = "Data_Processed/Shared_Task_hin/"
MODEL_PATH = "Saved_Models/Shared_Task_hin/all_but_one/best_bert_bert_1_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 4.5],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a Be

In [11]:
metrics

{'Test_accuracy': 0.8075657894736843,
 'Test_mF1Score': 0.6925479304464182,
 'Test_f1Score': 0.5045199089213626,
 'Test_auc': 0.6765116483215595,
 'Test_precision': 0.5934296330052435,
 'Test_recall': 0.43944878866091974,
 'Test_non_hatef1Score': 0.8805759519714739,
 'Test_non_recallScore': 0.9135745079821993,
 'Test_non_precisionScore': 0.849937755390578,
 'Test_avg_loss': 0.9881924117866315}

### Bengali 

In [12]:
DATA_PATH = "Data_Processed/Shared_Task_iben/"
MODEL_PATH = "Saved_Models/Shared_Task_iben/all_but_one/best_bert_bert_2_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 6.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a Be

In [13]:
metrics

{'Test_accuracy': 0.831418918918919,
 'Test_mF1Score': 0.6013067106508869,
 'Test_f1Score': 0.29842239545710997,
 'Test_auc': 0.5852102450277643,
 'Test_precision': 0.6443711725013197,
 'Test_recall': 0.1946606190089915,
 'Test_non_hatef1Score': 0.9041910258446638,
 'Test_non_recallScore': 0.9757598710465369,
 'Test_non_precisionScore': 0.8424198714551008,
 'Test_avg_loss': 0.8347795232727722}

### Spanish

In [14]:
DATA_PATH = "Data_Processed/AMI-Spanish/"
MODEL_PATH = "Saved_Models/AMI-Spanish/all_but_one/best_bert_bert_4_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a Be

In [15]:
metrics

{'Test_accuracy': 0.5631250000000001,
 'Test_mF1Score': 0.5089978695689356,
 'Test_f1Score': 0.3460475547747173,
 'Test_auc': 0.5616713614753089,
 'Test_precision': 0.6797050391956052,
 'Test_recall': 0.23226659110634523,
 'Test_non_hatef1Score': 0.6719481843631536,
 'Test_non_recallScore': 0.8910761318442724,
 'Test_non_precisionScore': 0.5393421571499156,
 'Test_avg_loss': 1.243747164607048}

## English

In [16]:
DATA_PATH = "Data_Processed/Shared_Task_eng/"
MODEL_PATH = "Saved_Models/Shared_Task_eng/all_but_one/best_bert_bert_1_all.pt"

args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 8.0],
        'save_model': False,
        'model_save_path': '',
        'name': 'bert_one_shot',
        'isArabic': False,
    }

model = BERT(args)

metrics = one_shot_output(MODEL_PATH,DATA_PATH,model,args)

/home/mithundas/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a Be

In [17]:
metrics

{'Test_accuracy': 0.7884191176470589,
 'Test_mF1Score': 0.5624156594521211,
 'Test_f1Score': 0.24793997940400397,
 'Test_auc': 0.5563677411235332,
 'Test_precision': 0.32192214690827975,
 'Test_recall': 0.20163718032468267,
 'Test_non_hatef1Score': 0.8768913395002382,
 'Test_non_recallScore': 0.9110983019223834,
 'Test_non_precisionScore': 0.8451626277539559,
 'Test_avg_loss': 0.7881933271884918}

# Few Shot Learning

Here we are checking the performance of the model trained on the English Dataset on other Datasets and their translated versions with finetuning

## BERT Model

### Imports

In [4]:
# Core
import random

# Basics
import numpy as np
import pandas as pd
import torch

# Metrics
from sklearn.metrics import *

# Tokeniser
from transformers import BertTokenizer

# Utility
from tqdm import tqdm

# Dataloader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Scheduler
from transformers import get_linear_schedule_with_warmup

# Optimiser
from transformers import AdamW

# Model
from transformers import BertForSequenceClassification
import torch.nn as nn

### Main Class

In [5]:
class BERT_FewShot:
    def __init__(self,args):
        # fix the random
        random.seed(args['seed_val'])
        np.random.seed(args['seed_val'])
        torch.manual_seed(args['seed_val'])
        torch.cuda.manual_seed_all(args['seed_val'])
        
        # set device
        self.device = torch.device(args['device'])

        self.weights=args['weights']
        
        # initiliase tokeniser
        self.tokenizer = BertTokenizer.from_pretrained(args['bert_model'])

        self.model_save_path = args['model_save_path']
        self.name = args['name']
        
    ##-----------------------------------------------------------##
    ##----------------- Utility Functions -----------------------##
    ##-----------------------------------------------------------##
    def encode(self,data,max_len):
        input_ids = []
        attention_masks = []
        for sent in tqdm(data):
            # use in-built tokeniser of Bert
            encoded_dict = self.tokenizer.encode_plus(
                            sent,
                            add_special_tokens =True, # for [CLS] and [SEP]
                            max_length = max_len,
                            truncation = True,
                            padding = 'max_length',
                            return_attention_mask = True,
                            return_tensors = 'pt', # return pytorch tensors
            )
            input_ids.append(encoded_dict['input_ids'])
            # attention masks notify where padding has been added 
            # and where is the sentence
            attention_masks.append(encoded_dict['attention_mask'])
        
        return [input_ids,attention_masks]
    
    ##-----------------------------------------------------------##
    ##------------------ Dataloader -----------------------------##
    ##-----------------------------------------------------------##
    def get_dataloader(self,samples, batch_size,is_train=False):
        inputs,masks,labels = samples

        # Convert the lists into tensors.
        inputs = torch.cat(inputs, dim=0)
        masks = torch.cat(masks, dim=0)
        labels = torch.tensor(labels)

        # convert to dataset
        data = TensorDataset(inputs,masks,labels)

        if(is_train==False):
            # use random sampler for training to shuffle
            # train data
            sampler = SequentialSampler(data)
        else:
            # order does not matter for validation as we just 
            # need the metrics
            sampler = RandomSampler(data)  

        dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size,drop_last=True)

        return dataloader
    
    ##-----------------------------------------------------------##
    ##----------------- Training Utilities ----------------------##
    ##-----------------------------------------------------------## 
    def get_optimiser(self,learning_rate,model):
        # using AdamW optimiser from transformers library
        return AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8
                )
    
    def get_scheduler(self,epochs,optimiser,train_dl):
        total_steps = len(train_dl) * epochs
        return get_linear_schedule_with_warmup(optimiser, 
                num_warmup_steps = 0, 
                num_training_steps = total_steps)
    
    def evalMetric(self, y_true, y_pred, prefix):
        # calculate all the metrics and add prefix to them
        # before saving in dictionary
        accuracy = accuracy_score(y_true, y_pred)
        mf1Score = f1_score(y_true, y_pred, average='macro')
        f1Score = f1_score(y_true, y_pred)
        area_under_c = roc_auc_score(y_true, y_pred)
        recallScore = recall_score(y_true, y_pred)
        precisionScore = precision_score(y_true, y_pred)

        nonhate_f1Score = f1_score(y_true, y_pred, pos_label=0)
        non_recallScore = recall_score(y_true, y_pred, pos_label=0)
        non_precisionScore = precision_score(y_true, y_pred, pos_label=0)
        return {prefix+"accuracy": accuracy, prefix+'mF1Score': mf1Score, 
            prefix+'f1Score': f1Score, prefix+'auc': area_under_c,
            prefix+'precision': precisionScore, 
            prefix+'recall': recallScore, 
            prefix+'non_hatef1Score': nonhate_f1Score, 
            prefix+'non_recallScore': non_recallScore, 
            prefix+'non_precisionScore': non_precisionScore}
    
    ##-----------------------------------------------------------##
    ##---------------- Different Train Loops --------------------##
    ##-----------------------------------------------------------## 
    def evaluate(self,model,loader,which):
        # to evaluate model on test and validation set

        model.eval() # put model in eval mode

        # maintain total loss to save in metrics
        total_eval_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(loader):
            # separate input, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            with torch.no_grad(): # do not construct compute graph
                outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
            # output is always a tuple, thus we have to 
            # separate it manually
            loss = outputs[0]
            logits = outputs[1]

            # add the current loss
            # loss.item() extracts loss value as a float
            total_eval_loss += loss.item()

            # calculate true labels and convert it into numpy array
            b_y_true = b_labels.cpu().data.squeeze().numpy()
            
            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

        # calculate metrics
        metrics = self.evalMetric(y_true,y_pred,which+"_")

        # Calculate the average loss over all of the batches.
        avg_loss = total_eval_loss / len(loader)
        # add it to the metric
        metrics[which+'_avg_loss'] = avg_loss

        return metrics
    
    
    def run_train_loop(self,model,train_loader,optimiser,scheduler):

        model.train() # put model in train mode

        # maintain total loss to add to metric
        total_loss = 0

        # maintain predictions for each batch and calculate metrics
        # at the end of the epoch
        y_pred = np.zeros(shape=(0),dtype='int')
        y_true = np.empty(shape=(0),dtype='int')

        for batch in tqdm(train_loader):
            # separate inputs, labels and attention mask
            b_input_ids = batch[0].to(self.device)
            b_input_mask = batch[1].to(self.device)
            b_labels = batch[2].to(self.device)

            # Ref: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch#:~:text=In%20PyTorch%20%2C%20we%20need%20to,backward()%20call.
            model.zero_grad()                

            outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

            # outputs is always returned as tuple
            # Separate it manually
            logits = outputs[1]

            # define new loss function so that we can include
            # weights
            loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(
                        self.weights,dtype=torch.float)).to(self.device)
            
            loss = loss_fct(logits,b_labels)
            
            # calculate current loss
            # loss.item() extracts loss value as a float
            total_loss += loss.item()

            # Back-propagation
            loss.backward()

            # calculate true labels
            b_y_true = b_labels.cpu().data.squeeze().numpy()

            # calculate predicted labels by taking max of 
            # prediction scores
            b_y_pred = torch.max(logits,1)[1]
            b_y_pred = b_y_pred.cpu().data.squeeze().numpy()

            y_pred = np.concatenate((y_pred,b_y_pred))
            y_true = np.concatenate((y_true,b_y_true))

            # clip gradient to prevent exploding gradient
            # problems
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # gradient descent
            optimiser.step()
            
            # schedule learning rate accordingly
            scheduler.step()

        # calculate avg loss 
        avg_train_loss = total_loss / len(train_loader)

        # calculate metrics
        train_metrics = self.evalMetric(y_true,y_pred,"Train_")
        
        # print results
        print('avg_train_loss',avg_train_loss)
        print('train_f1Score',train_metrics['Train_f1Score'])
        print('train_accuracy',train_metrics['Train_accuracy'])

        # add loss to metrics
        train_metrics['Train_avg_loss'] = avg_train_loss

        return train_metrics
    
    
    ##------------------------------------------------------------##
    ##----------------- Main Train Loop --------------------------##
    ##------------------------------------------------------------##
    def train(self,model,data_loaders,optimiser,scheduler,epochs,save_model):
        # save train stats per epoch
        train_stats = []
        test_stats=[]
        best_test={}
        train_loader,val_loader,test_loader = data_loaders
        # maintain best mF1 Score to save best model
        best_mf1Score=-1.0
        for epoch_i in range(0, epochs):
            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            
            print("")
            print('Training...')
            # run trian loop
            train_metrics = self.run_train_loop(model,train_loader,
                                            optimiser,scheduler)

            print("")
            print("Running Validation...") 
            # test on validation set
            val_metrics = self.evaluate(model,val_loader,"Val")
            
            print("Validation Loss: ",val_metrics['Val_avg_loss'])
            print("Validation Accuracy: ",val_metrics['Val_accuracy'])
            
            stats = {}
            
            if(val_metrics['Val_mF1Score']>best_mf1Score):
                print("Best mF1Score....")
                best_mf1Score=val_metrics['Val_mF1Score']
                if(save_model):
                    torch.save(model.state_dict(), self.model_save_path+
                        '/best_bert_'+self.name+'.pt')
                # evaluate model on test set
                best_test = self.evaluate(model,test_loader,'')
                best_test['name']=self.name+'_'+str(epoch_i)+'_best'


            stats['epoch']=epoch_i+1

            # add train and val metrics of the epoch to 
            # same dictionary
            stats.update(train_metrics)
            stats.update(val_metrics)

            train_stats.append(stats)
            
        test_stats.append(best_test)
        return train_stats,best_test
    
    ##-----------------------------------------------------------##
    ##----------------------- Main Pipeline ---------------------##
    ##-----------------------------------------------------------##
    def run(self,args,df_train,df_val,df_test):
        # get X and Y data points 
        X_train = df_train['Text'].values
        Y_train = df_train['Label'].values
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values
        X_val = df_val['Text'].values
        Y_val = df_val['Label'].values
        
        # encode data
        # returns list of data and attention masks
        train_data = self.encode(X_train,args['max_len'])
        val_data = self.encode(X_val,args['max_len'])
        test_data = self.encode(X_test,args['max_len'])
        
        # add labels to data so that we can send them to
        # dataloader function together
        train_data.append(Y_train)
        val_data.append(Y_val)
        test_data.append(Y_test)
        
        # convert to dataloader
        train_dl =self.get_dataloader(train_data,args['batch_size'],True)
        val_dl =self.get_dataloader(val_data,args['batch_size'])                          
        test_dl =self.get_dataloader(test_data,args['batch_size'])
        
        # intialise model
#         model = BertForSequenceClassification.from_pretrained(
#                 args['bert_model'], 
#                 num_labels = 2, 
#                 output_attentions = False, # Whether the model returns attentions weights.
#                 output_hidden_states = False, # Whether the model returns all hidden-states.
#             )
        model = self.load_model(args['model_path'],args)
        model.to(self.device)
        
        optimiser = self.get_optimiser(args['learning_rate'],model)
        
        scheduler = self.get_scheduler(args['epochs'],optimiser,train_dl)
        
        # Run train loop and evaluate on validation data set
        # on each epoch. Store best model from all epochs 
        # (best mF1 Score on Val set) and evaluate it on
        # test set
        train_stats,best_test = self.train(model,[train_dl,val_dl,test_dl],
                                optimiser,scheduler,args['epochs'],args['save_model'])
        
        return train_stats,best_test
        
    ##-----------------------------------------------------------##
    ##-------------------- Other Utilities ----------------------##
    ##-----------------------------------------------------------##
    def run_test(self,model,df_test,args):
        # to evaluate test set on the final saved model
        # to retrieve results if necessary
        X_test = df_test['Text'].values
        Y_test = df_test['Label'].values

        test_data = self.encode(X_test,args['max_len'])

        test_data.append(Y_test)

        test_dl =self.get_dataloader(test_data,32)

        metrics = self.evaluate(model,test_dl,"Test")

        return metrics
    
    def load_model(self,path,args):
        # load saved best model
        saved_model = BertForSequenceClassification.from_pretrained(
                args['bert_model'], 
                num_labels = 2, 
                output_attentions = False, # Whether the model returns attentions weights.
                output_hidden_states = False, # Whether the model returns all hidden-states.
            )
        
        saved_model.load_state_dict(torch.load(path))
        
        return saved_model

### Utility Functions

In [6]:
def load_dataset(args,index):
    # initialise constants 
    path = args['data_path']
    # read dataframes
    df_train = pd.read_csv(path+'train_'+str(index)+'.csv')
    df_val = pd.read_csv(path+'val_'+str(index)+'.csv')
    df_test = pd.read_csv(path+'test_'+str(index)+'.csv')

    # clean data
    df_train=preprocess(df_train,args['isArabic'])
    df_val=preprocess(df_val,args['isArabic'])
    df_test=preprocess(df_test,args['isArabic'])

    return df_train, df_val, df_test

In [7]:
def preprocess(df,isArabic):
    
    X = df['Text']
    X_new=[]
    if(isArabic):
        prep = ArabertPreprocessor('bert-base-arabertv02')
        for text in tqdm(X):
            text = prep.preprocess(text)
            X_new.append(text)
    else:
        processer = Data_Preprocessing()
        for text in tqdm(X):
            text= processer.removeEmojis(text)
            text = processer.removeUrls(text)
            text=processer.removeSpecialChar(text)
            X_new.append(text)

    df['Text']=X_new
    return df 

In [8]:
def save_metrics(path,metrics,which):
    df = pd.DataFrame(metrics)
    df.to_csv(path+"_"+which+".csv")

In [9]:
def fix_random(seed_val=42):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

### Main Train Function

In [10]:
def train(args, index,all_test_metrics,model_args):
    model_name = args['model_name']
    model_args['name']=model_name+'_'+str(index)+'_all'
    print("\tInitialising Model....")
    model = BERT_FewShot(model_args)
    print("\tLoading Dataset....")
    df_train, df_val, df_test = load_dataset(args,index)
    print("\tTraining Starts....")
    train_metrics, test_metrics = model.run(model_args, 
                    df_train, df_val, df_test)

    # Save train metrics after generating path
    res_path=args['res_base_path']+model_name+'_'+model_args['name']
    save_metrics(res_path,train_metrics,"train")
    
    all_test_metrics.append(test_metrics)

### Main Run Function

In [11]:
def run(args,model_args):
    all_test_metrics=[]
    
    for fold in [1, 2, 3, 4, 5]:
        print("Fold: ",fold)
        fix_random()
        train(args,fold,all_test_metrics,model_args)
        print("Saving Test Metrics....")
        save_metrics(args['res_base_path']+args['model_name']+
             '_all',all_test_metrics,"test")

## Arabic

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Let-Mi/',
    'train_cnt':256,
    'res_base_path': 'Results/Let-Mi/all_but_one/',
    'model_save_path': 'Saved_Models/Let-Mi/',
    'isArabic': True,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': True,
        'model_path': "Saved_Models/Let-Mi/all_but_one/best_bert_bert_3_all.pt",
    }
run(run_args,model_args)

Fold:  1
	Initialising Model....


/home/mithun-binny/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
  0%|          | 0/3670 [00:00<?, ?it/s]

	Loading Dataset....


  8%|▊         | 276/3670 [00:00<00:02, 1563.05it/s]

	Training Starts....


100%|██████████| 1047/1047 [00:00<00:00, 2508.89it/s]
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenc


======== Epoch 1 / 10 ========

Training...


  6%|▌         | 28/458 [00:04<01:04,  6.71it/s]

## Spanish

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-Spanish/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-Spanish/fewShot/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run(run_args,model_args)

Fold:  1
	Initialising Model....


 10%|█         | 235/2317 [00:00<00:00, 2347.45it/s]

	Loading Dataset....


 27%|██▋       | 634/2317 [00:00<00:00, 3140.35it/s]

	Training Starts....


...

  0%|          | 1/289 [00:00<00:57,  5.02it/s]


======== Epoch 3 / 10 ========

Training...


  5%|▍         | 2/41 [00:00<00:01, 19.52it/s]

avg_train_loss 0.5192106330064754
train_f1Score 0.7799747580984433
train_accuracy 0.7737889273356401

Running Validation...


 78%|███████▊  | 32/41 [00:01<00:00, 19.92it/s]

## Hindi

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Shared_Task_hin/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_hin/fewShot/',
    'model_save_path': 'Saved_Models/Shared_Task_hin/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 4.5],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng_1/best_bert_3_all.pt",
    }
run(run_args,model_args)

Fold:  1
	Initialising Model....


  7%|▋         | 285/4327 [00:00<00:01, 2839.92it/s]

	Loading Dataset....


 17%|█▋        | 749/4327 [00:00<00:00, 3752.48it/s]

	Training Starts....


...

  3%|▎         | 5/154 [00:00<00:03, 45.14it/s]

Validation Loss:  0.47130339979738384
Validation Accuracy:  0.8814935064935064
Testing Model....


  0%|          | 1/540 [00:00<01:42,  5.28it/s]


======== Epoch 5 / 10 ========

Training...


 24%|██▍       | 130/540 [00:26<01:23,  4.93it/s]

## Less data points few Shot

In [12]:
def load_dataset_part(train_cnt,args,index,seed):
    # initialise constants 
    path = args['data_path']
    # read dataframes
    df_train = pd.read_csv(path+'train_'+str(index)+'.csv')
    df_val = pd.read_csv(path+'val_'+str(index)+'.csv')
    df_test = pd.read_csv(path+'test_'+str(index)+'.csv')
    
    # split train into hate and non-hate and take train_cnt
    # samples of each
    df_train_hate = df_train[df_train['Label'] == 1].sample(train_cnt,random_state=seed)
    df_train_non_hate = df_train[df_train['Label'] == 0].sample(train_cnt,random_state=seed)
    # concatenate hate and non_hate
    df_train = pd.concat([df_train_hate, df_train_non_hate])
    # shuffle the train data
    df_train = df_train.sample(frac=1).reset_index(drop=True)

    # clean data
    df_train=preprocess(df_train,args['isArabic'])
    df_val=preprocess(df_val,args['isArabic'])
    df_test=preprocess(df_test,args['isArabic'])

    return df_train, df_val, df_test

In [13]:
def train_part(args,train_cnt,run,index,all_test_metrics,model_args,seed):
    model_name = args['model_name']
    model_args['name']=model_name+'_'+str(index)+'_'+str(train_cnt)+'_'+str(run)
    print("\tInitialising Model....")
    model = BERT_FewShot(model_args)
    print("\tLoading Dataset....")
    df_train, df_val, df_test = load_dataset_part(train_cnt,args,index,seed)
    print("\tTraining Starts....")
    train_metrics, test_metrics = model.run(model_args, 
                    df_train, df_val, df_test)

    # Save train metrics after generating path
    res_path=args['res_base_path']+model_name+'_'+model_args['name']
    save_metrics(res_path,train_metrics,"train")
    
    all_test_metrics.append(test_metrics)

In [14]:
def run_part(run_args,model_args,train_cnt):
    all_test_metrics=[]
    seeds = [42,43,44]
    for fold in [1, 2, 3, 4, 5]:
        print("Fold: ",fold)
        for run in [1,2,3]:
            print("Run: ",run)
            fix_random()
            train_part(run_args,train_cnt,run,fold,all_test_metrics,model_args,seeds[run-1])
            print("Saving Test Metrics....")
            save_metrics(run_args['res_base_path']+run_args['model_name']+
             '_'+str(train_cnt),all_test_metrics,"test")

## Arabic few data

In [18]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Let-Mi/',
    'train_cnt':256,
    'res_base_path': 'Results/Let-Mi/all_but_one/',
    'model_save_path': 'Saved_Models/Let-Mi/',
    'isArabic': True,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': True,
        'model_path': "Saved_Models/Let-Mi/all_but_one/best_bert_bert_3_all.pt",
    }
for train_cnt in [512]:
    print("Train Cnt: ",train_cnt)
    run_part(run_args,model_args,train_cnt)

Train Cnt:  512
Fold:  1
Run:  1
	Initialising Model....


KeyboardInterrupt: 

## Italian few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-2020/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-2020/all_but_one/',
    'model_save_path': 'Saved_Models/AMI-2020/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/AMI-2020/all_but_one/best_bert_bert_1_all.pt",
    }
for train_cnt in [64,128,256,512]:
    print("Train Cnt: ",train_cnt)
    run_part(run_args,model_args,train_cnt)

Train Cnt:  64
Fold:  1
Run:  1
	Initialising Model....


/home/mithun-binny/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
  0%|          | 0/128 [00:00<?, ?it/s]

	Loading Dataset....


 51%|█████     | 502/991 [00:00<00:00, 2444.09it/s]

	Training Starts....


100%|██████████| 1983/1983 [00:00<00:00, 2449.21it/s]
/home/mithun-binny/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


## Spanish Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/AMI-Spanish/',
    'train_cnt':256,
    'res_base_path': 'Results/AMI-Spanish/all_but_one/',
    'model_save_path': 'Saved_Models/AMI-Spanish/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/AMI-Spanish/all_but_one/best_bert_bert_4_all.pt",
    }
for train_cnt in [128,256,512]:
    print("Train Cnt: ",train_cnt)
    run_part(run_args,model_args,train_cnt)

Train Cnt:  128
Fold:  1
Run:  1
	Initialising Model....


/home/mithun-binny/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
  0%|          | 0/256 [00:00<?, ?it/s]

	Loading Dataset....


 61%|██████    | 202/330 [00:00<00:00, 2018.97it/s]

	Training Starts....


...

  7%|▋         | 3/41 [00:00<00:01, 23.01it/s]

avg_train_loss 0.3244188553071581
train_f1Score 0.8915187376725838
train_accuracy 0.892578125

Running Validation...


  4%|▎         | 3/82 [00:00<00:03, 22.98it/s]

Validation Loss:  1.066000823908281
Validation Accuracy:  0.7195121951219512
Best mF1Score....


 33%|███▎      | 27/82 [00:01<00:02, 23.00it/s]

## Hindi Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Shared_Task_hin/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_hin/all_but_one/',
    'model_save_path': 'Saved_Models/Shared_Task_hin/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_hin/all_but_one/best_bert_bert_1_all.pt",
    }
for train_cnt in [512]:
    print("Train Cnt: ",train_cnt)
    run_part(run_args,model_args,train_cnt)

Train Cnt:  512
Fold:  1
Run:  1
	Initialising Model....


## Bengali Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Shared_Task_iben/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_iben/all_but_one/',
    'model_save_path': 'Saved_Models/Shared_Task_iben/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_iben/all_but_one/best_bert_bert_2_all.pt",
    }
for train_cnt in [32,64,128,256,512]:
    print("Train Cnt: ",train_cnt)
    run_part(run_args,model_args,train_cnt)

## English Few Data

In [ ]:
run_args={
    'model_name':'few_shot',
    'data_path':'Data_Processed/Shared_Task_eng/',
    'train_cnt':256,
    'res_base_path': 'Results/Shared_Task_eng/all_but_one/',
    'model_save_path': 'Saved_Models/Shared_Task_eng/',
    'isArabic': False,
}

model_args={
        'seed_val': 42,
        'batch_size': 8,
        'bert_model': "bert-base-multilingual-cased",
        'learning_rate': 2e-5,
        'epochs': 10,
        'max_len': 128,
        'device': 'cuda:1',
        'weights': [1.0, 1.0],
        'save_model': False,
        'model_save_path': '',
        'isArabic': False,
        'model_path': "Saved_Models/Shared_Task_eng/all_but_one/best_bert_bert_1_all.pt",
    }
for train_cnt in [32,64,128,256,512]:
    print("Train Cnt: ",train_cnt)
    run_part(run_args,model_args,train_cnt)